In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math
import uuid

# إعداد Mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_draw = mp.solutions.drawing_utils

# الكاميرا
cap = cv2.VideoCapture(0)



# النقاط والرسوم
points = {}
current_trail = []
draw_mode = False
erasing = False
selected_point_id = None

def distance(p1, p2):
    return math.hypot(p1[0]-p2[0], p1[1]-p2[1])

def is_fingers_up(hand_landmarks):
    fingers = []
    landmarks = hand_landmarks.landmark

    fingers.append(landmarks[8].y < landmarks[6].y)  # سبابة
    fingers.append(landmarks[12].y < landmarks[10].y)  # وسطى
    fingers.append(landmarks[16].y < landmarks[14].y)  # بنصر
    fingers.append(landmarks[20].y < landmarks[18].y)  # خنصر
    fingers.append(landmarks[4].x < landmarks[3].x) if landmarks[4].x < landmarks[3].x else fingers.append(False)  # إبهام

    return fingers

def draw_trails(frame):
    for point_id, data in points.items():
        color = (255, 255, 0) if selected_point_id == point_id else (255, 255, 255)
        for i in range(1, len(data["trail"])):
            cv2.line(frame, data["trail"][i - 1], data["trail"][i], (255, 0, 0), 3)
        cv2.circle(frame, data["pos"], 10, color, -1)

# عامل تحويل بكسل إلى سم (بشكل تقريبي – يحتاج للمعايرة)
PIXEL_TO_CM = 0.026  # مثلًا: كل 38 بكسل ≈ 1 سم → 1/38 ≈ 0.026

while True:
    ret, frame = cap.read()
    if not ret:
        break
    h, w, _ = frame.shape
    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            lm = hand_landmarks.landmark
            fingers = is_fingers_up(hand_landmarks)

            index_finger = int(lm[8].x * w), int(lm[8].y * h)
            middle_finger = int(lm[12].x * w), int(lm[12].y * h)
            thumb_tip = int(lm[4].x * w), int(lm[4].y * h)

            # رسم بالإصبعين فقط (سبابة + وسطى)
            if fingers[0] and fingers[1] and not any(fingers[2:]):
                draw_mode = True
                current_trail.append(index_finger)
                cv2.circle(frame, index_finger, 5, (255, 0, 0), -1)

            # غلق اليد بالكامل = تسجيل كشكل دائري
            elif not any(fingers):
                if len(current_trail) >= 10:
                    center_x = sum([p[0] for p in current_trail]) / len(current_trail)
                    center_y = sum([p[1] for p in current_trail]) / len(current_trail)
                    center = (int(center_x), int(center_y))
                    avg_radius = sum([distance(p, center) for p in current_trail]) / len(current_trail)
                    variation = sum([abs(distance(p, center)-avg_radius) for p in current_trail]) / len(current_trail)
                    if variation < 20:
                        point_id = str(uuid.uuid4())
                        points[point_id] = {"pos": center, "trail": current_trail.copy()}
                current_trail = []
                draw_mode = False

            # مسك نقطة (إبهام + سبابة مضمومة)
            elif distance(index_finger, thumb_tip) < 40:
                if selected_point_id is None:
                    for pid, pdata in points.items():
                        if distance(pdata["pos"], index_finger) < 40:
                            selected_point_id = pid
                            break
                elif selected_point_id:
                    offset = (index_finger[0] - points[selected_point_id]["pos"][0],
                              index_finger[1] - points[selected_point_id]["pos"][1])
                    points[selected_point_id]["pos"] = index_finger
                    points[selected_point_id]["trail"] = [(p[0]+offset[0], p[1]+offset[1]) for p in points[selected_point_id]["trail"]]

            # اليد مفتوحة بالكامل = امسح أي نقطة قريبة
            elif all(fingers):
                to_delete = []
                for pid, pdata in points.items():
                    if distance(pdata["pos"], index_finger) < 40:
                        to_delete.append(pid)
                for pid in to_delete:
                    del points[pid]
                selected_point_id = None

            else:
                selected_point_id = None

            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # رسم الأثر والنقاط
    draw_trails(frame)

    if draw_mode:
        for i in range(1, len(current_trail)):
            cv2.line(frame, current_trail[i - 1], current_trail[i], (100, 100, 255), 2)

    # إذا كان في نقطتين، احسب المسافة بينهم بالسم
    if len(points) >= 2:
        pts = list(points.values())
        p1, p2 = pts[0]["pos"], pts[1]["pos"]
        dist_pixels = distance(p1, p2)
        dist_cm = dist_pixels * PIXEL_TO_CM
        mid = ((p1[0]+p2[0])//2, (p1[1]+p2[1])//2)
        cv2.putText(frame, f"{dist_cm:.2f} cm", mid, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
        cv2.line(frame, p1, p2, (0,255,0), 2)

    cv2.imshow("Smart Draw", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
